In [1]:
"""
Created on Thu Jun 16 11:19:17 2022
developed by Felix Panitz* and Peter Stange*
* at Chair of Building Energy Systems and Heat Supply, Technische Universität Dresden
"""
# TODO:
    #  Effektbeispiel mit Anzahl (z.b. Kesselanzahl < 3) 

'\nCreated on Thu Jun 16 11:19:17 2022\ndeveloped by Felix Panitz* and Peter Stange*\n* at Chair of Building Energy Systems and Heat Supply, Technische Universität Dresden\n'

In [2]:
useRohr = True
# useRohr = False
useAdditionalSink1 = True
# useAdditionalSink1 = False

In [3]:
### Inputs: ####
# Solver-Inputs:
displaySolverOutput = True  # ausführlicher Solver-Output.
gapFrac = 0.0001
timelimit = 3600

In [4]:
# solver_name = 'glpk'
# solver_name = 'gurobi'
solver_name    = 'cbc'
nrOfThreads    = 1

In [5]:
### Durchführungs-Optionen: ###
# doSegmentedCalc = True
doSegmentedCalc  = False
checkPenalty    = False  
excessCosts = None
excessCosts = 1e5 # default vlaue
################

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import datetime

In [7]:
from flixOpt.flixStructure import *
from flixOpt.flixComps import *

###################### kleine Daten zum Test ###############################

In [8]:
nrOfTimeSteps = 4
sink1 = [0.,   0.,  0., 100 , 40 , 40, 40, 40, 40][:nrOfTimeSteps]
sink2 = [10., 20., 30., 0   , 40 , 40, 40, 40, 40][:nrOfTimeSteps]

In [9]:
# todo: ggf. Umstellung auf numpy: aTimeSeries = datetime.datetime(2020, 1,1) +  np.arange(len(Q_th_Last)) * datetime.timedelta(hours=1)
aTimeSeries = datetime.datetime(2020, 1,1) +  np.arange(len(sink1)) * datetime.timedelta(hours=1)
aTimeSeries = aTimeSeries.astype('datetime64')

# DAtenreihen kürzen:
source1 = P_el_Last[0:nrOfTimeSteps]
sink1 = Q_th_Last[0:nrOfTimeSteps]
source2      = p_el     [0:nrOfTimeSteps]
aTimeSeries = aTimeSeries[0:nrOfTimeSteps]
#########################################################################

In [10]:
print('#######################################################################')
print('################### start of modeling #################################')

#######################################################################
################### start of modeling #################################


In [11]:
# Bus-Definition:
#                 Typ         Name              
heat1 = cBus('heat'        ,'heat1'     , excessCostsPerFlowHour = excessCosts);
heat2 = cBus('heat'        ,'heat2'     , excessCostsPerFlowHour = excessCosts);

In [12]:
# Effect-Definition:
costs = cEffectType('costs','€'      , 'Kosten', isStandard = True, isObjective = True)

In [13]:
aSink1   = cSink   ('Sink1'  ,sink   = cFlow('Q_th' , bus = heat1, nominal_val = 1, val_rel = sink1))
aSink2   = cSink   ('Sink2'  ,sink   = cFlow('Q_th' , bus = heat2, nominal_val = 1, val_rel = sink2))
aSource1 = cSource ('Source1',source = cFlow('Q_th' , bus = heat1, nominal_val = 60, costsPerFlowHour = -1))
aSource2 = cSource ('Source2',source = cFlow('Q_th' , bus = heat2, nominal_val = 60, costsPerFlowHour = -1)) # doppelt so teuer

In [14]:
loss_abs = 1
# loss_abs = 0
loss_rel = 0.1
# loss_rel = 0

In [15]:
invest1 = cInvestArgs(fixCosts=10,
                      investmentSize_is_fixed=False,
                      investment_is_optional=True,                      
                      max_investmentSize=1000 ,
                      specificCosts=1)

In [16]:
# only for getting realizing investSize-Variable:
invest2 = cInvestArgs(fixCosts=0,
                      investmentSize_is_fixed=False,
                      investment_is_optional=True,                      
                      max_investmentSize=1000 ,
                      specificCosts=0
                      )

In [17]:
aTransporter = cTransportation('Rohr', 
                                in1  = cFlow('in1',  bus=heat1, investArgs=invest1, nominal_val = None, min_rel = 0.1 ),
                                out1 = cFlow('out1', bus=heat2),
                                loss_abs = loss_abs,
                                loss_rel = loss_rel,
                                in2  = cFlow('in2',  bus=heat2, investArgs=invest2, nominal_val = None, min_rel = 0.1 ),
                                out2 = cFlow('out2', bus=heat1),
                                )

In [18]:
# Built energysystem:
es = cEnergySystem(aTimeSeries, dt_last=None)
# es.addComponents(aGaskessel,aWaermeLast,aGasTarif)#,aGaskessel2)
es.addEffects(costs)
es.addComponents(aSink2, aSource1, aSource2)
if useAdditionalSink1 : es.addComponents(aSink1)
if useRohr : es.addComponents(aTransporter)

Register new effect costs
Register new Component Sink2
Register new Component Source1
Register new Component Source2
Register new Component Sink1
Register new Component Rohr


In [19]:
chosenEsTimeIndexe = None
# chosenEsTimeIndexe = [1,3,5]

In [20]:
## modeling "full":
aCalc = cCalculation('Sim1', es, 'pyomo', chosenEsTimeIndexe)
aCalc.doModelingAsOneSegment()

finalize all MEs...
Sink2
Source1
Source2
Sink1
Rohr
heat1
heat2
globalComp
costs
out2
Q_th
Q_th
Q_th
Q_th
in1
in2
out1
modeling costs
declareVarsAndEqs Q_th
declareVarsAndEqs Q_th
declareVarsAndEqs Q_th
declareVarsAndEqs Q_th
declareVarsAndEqs in1
declareVarsAndEqs in2
declareVarsAndEqs out1
declareVarsAndEqs out2


In [21]:
# PRINT Model-Charactaricstics:
es.printModel()
es.printVariables()
es.printEquations()


##############################################################
########## Short String Description of Energysystem ##########

buses:
  heat1:
    In-Flows:
    - Q_th:
        comp: Source1
    - out2:
        comp: Rohr
    Out-Flows:
    - Q_th:
        comp: Sink1
    - in1:
        comp: Rohr
    class: cBus
  heat2:
    In-Flows:
    - Q_th:
        comp: Source2
    - out1:
        comp: Rohr
    Out-Flows:
    - Q_th:
        comp: Sink2
    - in2:
        comp: Rohr
    class: cBus
components:
  Rohr:
    In-Flows:
    - in1:
        bus: heat1
    - in2:
        bus: heat2
    Out-Flows:
    - out1:
        bus: heat2
    - out2:
        bus: heat1
    class: cTransportation
  Sink1:
    In-Flows:
    - Q_th:
        bus: heat1
    Out-Flows: []
    class: cSink
  Sink2:
    In-Flows:
    - Q_th:
        bus: heat2
    Out-Flows: []
    class: cSink
  Source1:
    In-Flows: []
    Out-Flows:
    - Q_th:
        bus: heat1
    class: cSource
  Source2:
    In-Flows: []
    Ou

In [22]:
solverProps = {'gapFrac': gapFrac, 
               'timelimit': timelimit,
               'solver': solver_name, 
               'displaySolverOutput' : displaySolverOutput,
               }
if solver_name == 'gurobi': solverProps['threads'] = nrOfThreads

# calculation "full":

In [23]:
aCalc.solve(solverProps, nameSuffix = '_' + solver_name)


##############################################################
##################### solving ################################

no of Eqs   (single):30 (48)
no of InEqs (single):14 (44)
no of Vars  (single):45 (93)
Solver log file: '/Users/felix/Documents/Dokumente - eigene/Arbeit_SE_Trainee/flixOptSE/flixOpt/examples_notebooks/results/2024-05-25_Sim1_cbc_solver.log'
Solver solution file: '/var/folders/1z/xs11s8cd563_dhtnmk53bxpr0000gn/T/tmphhlct_r7.pyomo.soln'
Solver problem files: ('/var/folders/1z/xs11s8cd563_dhtnmk53bxpr0000gn/T/tmphhlct_r7.pyomo.lp',)
Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Aug  1 2023 

command line - /Users/felix/opt/anaconda3/bin/cbc -ratio 0.0001 -sec 3600 -printingOptions all -import /var/folders/1z/xs11s8cd563_dhtnmk53bxpr0000gn/T/tmphhlct_r7.pyomo.lp -stat=1 -solve -solu /var/folders/1z/xs11s8cd563_dhtnmk53bxpr0000gn/T/tmphhlct_r7.pyomo.soln (default strategy 1)
ratioGap was changed from 0 to 0.0001
seconds was changed from 1e+100 to 3

In [24]:
aCalc.results_struct

In [25]:
print('source1 :' + str(aCalc.results_struct.Source1.Q_th.val))
print('source2 :' + str(aCalc.results_struct.Source2.Q_th.val))
# print(aCalc.results_struct.Sink2.Q_th.val)

source1 :[12.222222 23.333333 34.444444 60.      ]
source2 :[ 0.        0.        0.       45.555556]


In [26]:
if useRohr :
    print('rohr_in :' + str(aCalc.results_struct.Rohr.in1.val))
    print('rohr_out:' + str(aCalc.results_struct.Rohr.out1.val))

rohr_in :[12.222222 23.333333 34.444444  0.      ]
rohr_out:[10. 20. 30.  0.]


In [27]:
print(aCalc.infos['modboxes']['info'][0]['main_results']['Invest-Decisions'])

{'invested': {'Rohr__in1': 45.555556, 'Rohr__in2': 45.555556}, 'not invested': {}}


# calculation "segmented":